## FINANCIAL DATA

MODULE 3 | LESSON 4


---

# IDENTIFYING AND APPLYING DOWNSIDE RISK METRICS ASSOCIATED WITH FINANCIAL MARKETS

|  |  |
|:---|:---|
|**Reading Time** |  30 minutes |
|**Prior Knowledge** | Stock Indices, Log return calculations  |
|**Keywords** | Value at Risk, Expected Shortfall |

---

*This lesson will go over a fundamental risk concept, Value at Risk. This will allow us to estimate the maximum amount of money we can lose in a single day given a confidence level. This metric is employed at banks and financial institutions around the world in order to mitigate market risk. Using this and it's many variations will help us build a well-balanced portfolio to try to protect us from financial losses going forward*

In [ ]:
import datetime

import numpy as np
import pandas as pd
import pandas_datareader.data as web
import seaborn as sns
import yfinance as yfin
from IPython.display import VimeoVideo
from scipy import stats

yfin.pdr_override()

## 1. Value at Risk (VaR)

Value at Risk is one of the easiest risk metrics to interpret. So far, the metrics we have introduced quantify risk as a percentage, in the case of standard deviation, or in units, as the Sharpe ratio does. Value at Risk answers the fundamental question many investors have on their mind: How much can I lose on an investment in a worst-case scenario? This is measured in dollars for the purposes of this class.

Value at Risk measures the potential loss in value of an asset/portfolio over a defined time period. Basically, you will always need to specify the time period and confidence interval that accompanies a Value at Risk. For example, if the VaR of a portfolio is \\$1,000,000 over a yearly time period with a 99\% confidence interval, it would mean that the portfolio only has a 1\% chance of losing more than \\$1,000,000 for any given year. VaR has become ubiquitous over the years; every investment bank and risk management firm is employing some form of VaR to help keep a cap on the potential losses one can incur. The focus on VaR is very much about downside risk unlike something like standard deviation, which looks at both the upside and downside risk.

There are three basic methods for calculating VaR, each with their own advantages and disadvantages. These should build on some of the lessons from earlier, like variance and covariance. Keep in mind that there are countless variations of each basic method, but we will stick with these main three for now:

* Historical Method
* Parametric Method 
* Monte Carlo Simulation


## 2. Historical Method

This is probably the simplest and most intuitive method of calculating Value at Risk. In short, historical returns are sorted from lowest to highest on an asset or portfolio. Let’s say you wanted to calculate the daily Value at Risk on an equity with a 95% confidence interval. Assuming we can look at the last thousand days of data for this stock, we would take the daily returns and sort them from lowest to highest. From here, we would take the return from the 5th percentile of the data. In this case, with 1,000 days of data, it would be the 50th (0.05*1000) worst daily return from these thousand days. Let’s say the 50th worst day had a –4% return. From this, we can assume that the daily VaR for this stock with a 95% confidence interval is –4%. Building on that, if we were to invest \$1,000 in said stock, we would expect the worst daily loss to be:

-0.04*1000 = -\$40  with a 95% confidence interval

Considerations for this method:

* This method uses historical returns to measure VaR empirically, which means that there are no assumptions made about the distribution whereas many models assume the normal distribution.
* Each day for this method is given equal weight, which means if there is a trend in the volatility, you could be overstating or understating the VaR depending on whether the volatility trend is down or up, respectively. One refinement to combat this could be to place greater weight on more recent data.
* Past data does not necessarily indicate what will happen in the future. While the other methods also rely on historical data to a certain extent, this method is solely derived from past historical returns. There are many unforeseen events that can happen, which can change the course of a stock’s trajectory and cause the stock to trade differently than it did in the past.



### 2.1 Implementing Value at Risk (VaR) - Historical Method

Calculating daily historical VaR can be done pretty simply in Python. The order of steps needed is as follows:
1. Calculate all daily returns.
2. Sort these returns from least to greatest.
3. Based on a given confidence level, return the corresponding percentile return. In other words, if we want to calculate daily VaR at a 95% confidence level and we are using 100 data points, the 5th smallest return of that sample would be considered our 95% VaR.

Let's visualize this first with a histogram using Bitcoin.

In [ ]:
start = datetime.date(2016, 1, 1)
end = datetime.date(2021, 11, 28)
# end = datetime.date.today()

prices = pd.DataFrame(web.DataReader(["BTC-USD"], start, end)["Adj Close"])
returns = prices.pct_change()
returns = returns.rename(columns={"Adj Close": "Bitcoin"})
returns = returns.dropna()

In [ ]:
sns.histplot(data=returns);

It may be a little hard to see, but the graph shows a lot more to the left of 0 than to the right of 0, indicating there may have been some big losses over the last 5 years. Let's calculate the historical VaR from this data.

We need to can use numpy's *percentile* method in order to easily calculate this for us. Using this function allows us to skip the sorting step as it is already factored into the logic. We can define a function below in order to look at various confidence intervals. This function `getHistoricalVar()` takes two arguments:
1. a series of daily returns
2. a confidence level, 95 for example, would correspond to 95% confidence

In [ ]:
def getHistoricalVar(returns, confidenceLevel):
    var = 100 * np.percentile(returns, 100 - confidenceLevel)
    print(
        "With %.2f%% percent confidence, we can say the most our \
        \nportfolio will lose in a day is %.3f%% using historical VaR"
        % (confidenceLevel, var)
    )

In [ ]:
getHistoricalVar(returns.Bitcoin, 95)

In [ ]:
getHistoricalVar(returns.Bitcoin, 99)

It can be seen after running the function that the greater we make our confidence level, the lower the Value at Risk will be. This is a useful tool when determining how much a financial asset can lose over a certain time period.

Using the historical method, if we have -10% as the 95% confidence VaR, it can be interpreted that 95% of the time, our asset shouldn't return lower than -10% in a single day. 

You may be wondering now, since we only use a 95% confidence level, what happens on those 5% of days where losses exceed -10%. We can use another handy metric, Conditional Value at Risk (CVaR), to deal with these situations.

## 3. Conditional Value at Risk (CVaR)
CVaR is also commonly known as expected shortfall. To calculate this, we look at all of the daily returns that are lower than our Value at Risk and take the average of those values. It's as simple as that. We can index into our Python returns series in order to calculate it. Once, again we will write a function in order to calculate this with various parameters.


In [ ]:
def getHistoricalCVar(returns, confidenceLevel):
    var = np.percentile(returns, 100 - confidenceLevel)
    cvar = returns[returns <= var].mean()
    print(
        "With %.2f%% percent confidence VaR, our Expected Shortfall is %.2f%% using historical VaR"
        % (confidenceLevel, 100 * cvar)
    )

In [ ]:
getHistoricalCVar(returns.Bitcoin, 95)

The above can be interpreted that in the 5% of days where returns are less than our VaR, we can expect our return to average about -9%. This is another helpful tool when assessing the risk of an asset. Since these functions are defined now, we can quickly compare the Bitcoin results to a bond index, BLV. 

In [ ]:
start = datetime.date(2016, 1, 1)
end = datetime.date(2021, 11, 28)
# end = datetime.date.today()

prices = pd.DataFrame(web.DataReader(["BLV"], start, end)["Adj Close"])
prices = prices.rename(columns={"Adj Close": "BLV"})
returns = prices.pct_change()
returns = returns.dropna()

In [ ]:
getHistoricalVar(returns.BLV, 95)

In [ ]:
getHistoricalCVar(returns.BLV, 95)

While Bitcoin's 95% VaR and CVaR over the last five years is about -6% and -9% respectively, the bond index, BLV, has -0.93% and -1.5% for the same metrics. This goes to show how much more downside risk an asset like Bitcoin holds when compared to bonds.

With the historical method, we're using only values that actually occurred in our dataset. This may not always be the scientific method because what happened over the last five years may not reflect what will happen over the next five years. We need to explore all of our options in order to best understand the risk profile of our investments. This leads us to a different way to calculate VaR. 

In the first video of this lesson, we recap the definition of VaR and how to calculate it using the historical method.

In [ ]:
VimeoVideo("706655901", h="21ae7573c3", width=600)

##### [Access video transcript here](https://drive.google.com/file/d/1_3Tz0fJAiAq_XS5q7kDE6vu4lKwVSyH6/view?usp=sharing)

## 4. Parametric Method
Since Value at Risk essentially is derived using probabilities to determine potential losses, we can use a probability distribution to compute VaR here. In the standard variation, a normal distribution of returns is assumed in the standard case. Just keep in mind that we can use other distributions to calculate this, as will be shown in the second example with a t-distribution. 

Example:

Assume you have an account with a mean value of \\$10,000 and an annual standard deviation of \\$500 and normally distributed returns. We can find that the critical value of a 95% confidence interval of a normal distribution is 1.96. Given these parameters, we can estimate with 95% probability that the most we can lose or gain in a year is: 500 * 1.96 = \\$980



Considerations for this method:

* If returns are not normally distributed, you will likely be underestimating the true Value at Risk. For example, many stocks have more outliers than the normal distribution would assume; this means the computed Value at Risk will be lower than what it is in actuality.
* Variance and covariance between return streams must be considered when computing VaR for a portfolio. Even if returns are normally distributed, the VaR calculation can still be thrown off if the estimated variances and covariances are incorrect. This can be further amplified if variances and covariances are changing over time.
* Models that allow variance to change over time (heteroskedasticity), display a greater degree of accuracy. Engle has argued that Autoregressive Conditional Heteroskedasticity (ARCH) and Generalized Autoregressive Conditional Heteroskedasticity (GARCH) models provide better forecasts of variance and, by extension, better measures of Value at Risk (Engel 157-168)
* This method breaks down whenever a portfolio has assets with a non-linear payoff structure, e.g., options.


### 4.1. Implementing VaR - Parametric Method with Normal Distribution
To calculate VaR using the parametric method is to assume a normal distribution for our returns and pass in the mean and standard deviation of our data into the `norm.ppf()` method. This is a percent point function and allows us to pass in a confidence interval along with a mean and standard deviation to compute VaR. Even though we discussed that stock returns are non-normal in a previous lesson, this method of VaR calculation is arguably still a more valuable estimate than just using historical returns and will give us greater flexibility when doing Monte Carlo simulations in the near future.

Let's use the AAPL data back to 2010 in order to calculate this. 

In [ ]:
start = datetime.date(2010, 1, 1)
end = datetime.date(2021, 11, 28)
# end = datetime.date.today()

prices = pd.DataFrame(web.DataReader(["AAPL"], start, end)["Adj Close"])
prices = prices.rename(columns={"Adj Close": "AAPL"})
returns = prices.pct_change()
returns = returns.dropna()

In [ ]:
mean = returns.AAPL.mean()
std = returns.AAPL.std()
(100 * stats.norm.ppf(0.05, mean, std)).round(3)

This method gives us a 95% confidence VaR of -2.78%. We can compare it to the historical method using our previous function.

In [ ]:
getHistoricalVar(returns.AAPL, 95)

It's quite interesting to note that, in this case, the parametric VaR gives a slightly more conservative estimate of daily VaR by having a lower expected return.

### 4.2 Implementing VaR - Parametric Method with T-distribution
We just need to define our degrees of freedom for this method, but besides that, we can plug in the same parameters from above into the t-distribution function. We will use four degrees of freedom in this case. The higher you make the degrees of freedom, the more conservative VaR estimate you will have.



In [ ]:
# degrees of freedom
dof = 4
tVaR = np.sqrt((dof - 2) / dof) * stats.t.ppf(0.05, dof) * std - mean
(100 * tVaR).round(3)

We would expect the results to be similar, but it is quite remarkable that the 95% VaR using a t-distribution matches with what we had using a normal distribution.

In the final video of this lesson, we show once again how to calculate VaR using the parametric method in Python.



In [ ]:
VimeoVideo("706655936", h="435573b3f4", width=600)

##### [Access video transcript here](https://drive.google.com/file/d/12HhJ2V31itS9f-AGXGl92LeUNEAAixeK/view?usp=sharing)

## 5. Monte Carlo Simulation

This method runs a series of simulations, usually in the thousands, where each return stream is represented as a random variable. This variable can be taken from any probability distribution, which is great because that means it doesn’t necessarily assume the normal distribution. There is a lot of flexibility in choosing what kind of distribution to use. All the variables are then dollar-weighted and simulated to see what the total portfolio value is at the end of each run. These simulation returns are then sorted lowest to highest, and we can easily look to see what the Value at Risk is using similar computations to the historical method, except this time, we’re using simulated returns instead of historical returns. For example, if you ran a series of 1,000 simulations, you would look at the 50th lowest value to determine the VaR for a 95% confidence interval.



Considerations for this method:
* Estimations will not be effective if the probability distributions used to determine the random variables are incorrect. Many use past data to get an idea of what the probability distribution should be; this method at least allows some subjectivity to doing this.
* You can estimate VaR more effectively for portfolios containing options with this method versus the parametric method since Monte Carlo doesn’t assume a normal distribution of returns.


## 6. Conclusion

Throughout this lesson, we've examined different methods to calculate downside risk metrics like Value at Risk and expected shortfall. These should help shape your risk preferences before you buy assets for your portfolio.

**References**

- Engle, R., 2001, Garch 101: The Use of ARCH and GARCH models in Applied Econometrics, Journal of Economic Perspectives, v15, 157-168.


---
Copyright 2023 WorldQuant University. This
content is licensed solely for personal use. Redistribution or
publication of this material is strictly prohibited.
